In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from unidecode import unidecode
from selenium.webdriver.common.keys import Keys
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime

driver = webdriver.Chrome('/Users/ik/Data/webdrivers/chromedriver')
WAIT_TIME = 30

In [9]:
emails = pd.read_csv("/Users/ik/Data/email-domains/verified_email_domains.txt", sep=":", header=None, names="domain status".split())

In [10]:
ok_emails = sorted(list(set(emails.loc[emails.status=='ok']["domain"].tolist())))
print("total ok emails: {}".format(len(ok_emails)))

total ok emails: 360181


In [11]:
for company_domain in ok_emails[:10]:
    
    print("searching for {}...".format(company_domain))
    
    comp_info = []  # list of all company information pieces
    
    driver.get("https://www.google.com.au/")
    SUBJ_WORD = " ".join(["linkedin", company_domain])
    WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, "//input[@id='lst-ib']"))).send_keys(SUBJ_WORD)
    driver.find_element_by_xpath("//input[@type='submit']").click()

    h3 = driver.find_element_by_xpath("//div[@class='s']")
    linkedin_url = h3.find_element_by_xpath(".//cite[@class='_Rm']").text
    
    if "au.linkedin.com" in linkedin_url:
        driver.get(linkedin_url)
        
        try:
            WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, "public-follow")))
        except:
            print("no follow button, skipping..")
            continue
            
        try:
            cname = driver.find_element_by_xpath("//h1[@class='name']")
        except:
            # we don't care about companies without names
            continue
        
        try:
            indus = driver.find_element_by_xpath("//p[@class='industry']")
        except:
            indus = None
            
        try:
            csize = driver.find_element_by_xpath("//p[@class='company-size']")
        except:
            size = None
            
        try:
            cwebs = driver.find_element_by_xpath("//li[@class='website']/p")
        except:
            cwebs = None
        
        try:
            ctype = driver.find_element_by_xpath("//li[@class='type']/p")
        except:
            ctype = None
        
        try:
            cadr = driver.find_element_by_xpath("//p[@class='adr']")
        except:
            cadr = None
            
        comp_info.append({k: v.text.lower().strip() if v else None for k, v in {"name": cname, "industry": indus, "size": csize,
                                "website": cwebs, "type": ctype, "address": cadr}.items()})
    
        print(comp_info)
        
    else:
        print("can\'t find this company on linkedin, moving on...".format(company_domain.upper()))     
    
driver.quit()

searching for 0014.com.au...
no follow button, skipping..
searching for 007designs.net...
[{'name': '.net professionals', 'industry': 'staffing and recruiting', 'size': '1-10 employees', 'website': 'http://www.net-professionals.nl', 'type': 'privately held', 'address': None}]
searching for 007security.com...
[{'name': '007 security', 'industry': 'security and investigations', 'size': '1-10 employees', 'website': 'http://007securitysolutions.ca/', 'type': None, 'address': '240 e 2nd st new york, new york 10009-8038 united states'}]
searching for 007security.com.au...
no follow button, skipping..
searching for 01.com.au...
no follow button, skipping..
searching for 010.com.au...
[{'name': '010 productions', 'industry': 'international trade and development', 'size': '11-50 employees', 'website': 'http://www.010productions.com', 'type': 'privately held', 'address': 'lee world building no. 57 xingfucun zhong lu, chaoyang district beijing, beijing 100027 china'}]
searching for 0101it.com...
